### A notebook to run the GW shapes for the GAB project

In [1]:
from datacube import Datacube
from datacube.virtual.impl import VirtualDatasetBox
from datacube.virtual import construct
from datacube.utils.geometry import CRS, Geometry
from datacube.utils.geometry.gbox import GeoboxTiles
import boto3
import os
import fiona
import yaml
import numpy as np
import pandas as pd
from datetime import datetime
from shapely import ops, geometry
import pyproj

In [2]:
from datacube_wps.processes.witprocess import *
from datacube.utils.dask import start_local_dask
#from datacube.utils.aws import configure_s3_access

In [3]:
client = start_local_dask(n_workers=1, threads_per_worker=7, memory_limit='58GB')

In [4]:
client

Client Scheduler: tcp://127.0.0.1:44627 Dashboard: /user/bex.dunn@ga.gov.au/proxy/8787/status,Cluster Workers: 1 Cores: 7 Memory: 58.00 GB


In [6]:
import awswrangler as wr

In [22]:
session = boto3.Session(profile_name='dev')
#session = boto3.Session()

ProfileNotFound: The config profile (dev) could not be found

In [12]:
landsat_shapefile = '../auxfiles/landsat_au/landsat_au.shp'

In [13]:
def shape_list(shapefile):
    with fiona.open(shapefile) as allshapes:
        for shape in allshapes:
            yield(shape)

In [14]:
def intersect_landsat_pathrow(input_shape, landsat_shapes):
    project = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3577", always_xy=True).transform
    input_poly = geometry.shape(input_shape['geometry'])
    for l_shape in landsat_shapes:
        overlap = ops.transform(project,  geometry.shape(l_shape['geometry'])).intersection(input_poly)
        if overlap.area > 0:
            if overlap.area / input_poly.area > 0.9:
                return(input_shape['id'], 1)
    return (input_shape['id'], 15)

In [16]:
    product_yaml = '../auxfiles/fc_pd_reproject.yaml'
    with open(product_yaml, 'r') as f:
        recipe = yaml.safe_load(f)
    fc_product = construct(**recipe)

In [17]:
config = yaml.load(""" 
about:
       identifier: FractionalCoverDrill
       version: '0.3'
       title: Fractional Cover
       abstract: Performs Fractional Cover Polygon Drill
       store_supported: True
       status_supported: True
       geometry_type: polygon
       """, Loader=yaml.CLoader)
wit = WIT(config['about'], fc_product, '')

mask all touch True


In [18]:
shapefile_name = '../auxfiles/GESIRA_modelv2.shp'
# shapefile_name = 'qld_shapefile/Queensland_dominant_wetland_areas_22042020.shp'

In [20]:
time = ('1987-01-01', '2021-01-01')
dc = Datacube()
i = 0
for shape in shape_list(shapefile_name):
    if int(shape['id']) < 67: #id is hard coded, check this here
        continue
    shape_set = intersect_landsat_pathrow(shape, shape_list(landsat_shapefile))
    print("compute", shape_set)
    query_poly = Geometry(shape['geometry'], crs=CRS('EPSG:3577'))
    print("start query time", datetime.now())
    %time results = wit.input_data(dc, time, query_poly)
    print("start computation", datetime.now())
    adays = shape_set[1]
    %time re_wit = wit.process_data(results, dict(feature=query_poly, aggregate=adays))
    print("end computation", datetime.now())
    re_wit['geometry'] = query_poly.geom.convex_hull.to_wkt()
    wr.s3.to_parquet(df=re_wit.reset_index(), 
            # path="s3://dea-wit-dev/c3-samples-3577/ramsar/"+shape['id'], compression="snappy", dataset=True, mode='overwrite',boto3_session=session)
              path="s3://dea-wit-dev/c3-samples-3577/GW/"+shape['id'], compression="snappy", dataset=True, mode='overwrite',boto3_session=session)

In [21]:
# gather and rewrite the parquet file to one single large

#need to put these in a different bucket
a = wr.s3.read_parquet("s3://dea-wit-dev/c3-samples-3577/GW/", dataset=True, boto3_session=session)
wr.s3.to_parquet(df=a, path="s3://dea-wit-dev/c3-samples-3577/GW/all/all.parquet", compression="snappy", dataset=False, boto3_session=session)

ClientError: An error occurred (InvalidAccessKeyId) when calling the ListObjectsV2 operation: The AWS Access Key Id you provided does not exist in our records.